If using Google Colab: <br>
Recap: Instructions for connecting to Google Drive here: https://medium.com/@wl8380/how-to-mount-google-drive-in-google-colab-c688ec8eccb7

Right Click on Files on the left, Add New Folder and Upload files to the drive.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

### Read here:

Geopandas: https://geopandas.org/en/stable/getting_started/introduction.html <br>
Shapely: Manipulation and Analysis of Planar Geometric Objects https://shapely.readthedocs.io/en/stable/

In [ ]:
# install whatever you need

In [ ]:
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt

#### Data Exploration

Data downloaded from here: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
file_path = 'yellow_tripdata_2024-01.parquet'

In [ ]:
df =

In [ ]:
df.info()

Already in datetime format

In [ ]:
df.head()

## Goal: Predicting number of taxi pickups in a given location

Spatial-temporal prediction (geographical and time-based factors)

#### Feature Engineering: Features that contribute to your model

1. Temporal Features: tpep_pickup_datetime

2. Location Features: PULocationID

3. Lagged Features : Past demand (number of pickups in previous hour at each location) <br>
Lagged Features: Created by taking the value of a variable at a previous time point and including it as a feature in the model at the current time point. Helps to capture temporal dependencies and patterns in time series data

#### 1. Cleaning your data

In [ ]:
# Create new temporal features
df['hour'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Add a 'day' feature for our temporal split later
df['day'] = df['tpep_pickup_datetime'].dt.day

In [ ]:
# Check df after manipulation
df

In [ ]:
# Aggregate the data to create our target variable 'pickup_count'
# We want to know the count of pickups for each location, at each day, at each hour.
df_pickups = df.groupby(['PULocationID', 'day', 'hour', 'day_of_week', 'is_weekend']).size().reset_index(name='pickup_count')
df_pickups.head()

#### 2. Preparing for Modeling

In [ ]:
features = ['PULocationID', 'hour', 'day_of_week', 'is_weekend']
target = 'pickup_count'

X = df_pickups[features + ['day']]
y = df_pickups[target]

In [ ]:
# Create a Temporal Split -  train on days 1-24 and test on days 25-31
test_day_start = 25
train_indices = X['day'] < test_day_start
test_indices = X['day'] >= test_day_start


In [ ]:
# Drop the 'day' column AFTER splitting, as it's not a predictive feature
X_train = X[train_indices].drop(columns=['day'])
y_train = y[train_indices]

In [ ]:
X_test = X[test_indices].drop(columns=['day'])
y_test = y[test_indices]

In [ ]:
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

#### 3. Train a Random Forest Regressor model

In [ ]:
print("\nTraining Model 1 (Simple)...")
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Evaluate the Model 
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModel 1 Evaluation ")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2) Score: {r2:.3f}")

MSE: This suggests that while your average error was low, you had a few predictions that were off by a larger amount, which is totally normal <br>
MAE: On average, the prediction is off by x pickups compared to the actual number of pickups. <br>
R2: Model explains around x% of variability in the pickup demand. Strong fit of model. <br>

Note: The above is a simplified version. The better way to improve this is to use a dynamic, lagged feature (i.e., the pickup count at this exact location, 24 hours ago. Simply put, it is able to capture momentum and real-world events and is a far more powerful model. But we will not be implementing this in the notebook.

Note 2: This is where you will spend time fine tuning your models etc.

#### 4. Adding Predictions to dataset for visualization

In [ ]:
X_test_with_predictions = X_test.copy()
X_test_with_predictions['actual_pickup_count'] = y_test
X_test_with_predictions['predicted_pickup_count'] = y_pred

In [ ]:
# Aggregate the results by location to get the average counts for mapping
df_map_data = X_test_with_predictions.groupby('PULocationID')[['actual_pickup_count', 'predicted_pickup_count']].mean().reset_index()

In [ ]:
# Let's also create an error column to see where the model was most wrong
df_map_data['prediction_error'] = df_map_data['actual_pickup_count'] - df_map_data['predicted_pickup_count']

print("Aggregated data for mapping (df_map_data):")
df_map_data.head()

Combining with latlon information found here: https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc

In [ ]:
file_path = 'nyc_taxi_zones.geojson'

In [ ]:
taxi_zones = 

In [ ]:
# If you are more comfortable you can do this in GIS as well

In [ ]:
# Check the Map projection. original CRS (it should be EPSG:4326)
print(f"Original CRS: {taxi_zones.crs}")

In [ ]:
# a. Project to a projected CRS (EPSG:32618)
taxi_zones_proj = taxi_zones.to_crs(epsg=32618)

In [ ]:
# b. Calculate the centroid on the projected data
taxi_zones_proj['centroid'] = taxi_zones_proj.geometry.centroid

In [ ]:
# c. Project the new centroid column back to Lat/Lon (EPSG:4326)
taxi_zones['centroid_latlon'] = taxi_zones_proj['centroid'].to_crs(epsg=4326)

In [ ]:
# d. Extract the correct latitude and longitude
taxi_zones['latitude'] = taxi_zones['centroid_latlon'].y
taxi_zones['longitude'] = taxi_zones['centroid_latlon'].x

In [ ]:
print("\nTaxi zones with centroids:")
print(taxi_zones[['location_id', 'zone', 'latitude', 'longitude']].head())

In [ ]:
# Create a clean lookup table for coordinates
taxi_zone_coords = taxi_zones[['location_id', 'latitude', 'longitude']].copy()

In [ ]:
taxi_zone_coords.loc[:, 'location_id'] = taxi_zone_coords['location_id'].astype('int32')

In [ ]:
print("\nCoordinate lookup table (taxi_zone_coords):")
print(taxi_zone_coords.info())

In [ ]:
# Create a Geodataframe
taxi_gdf['geometry'] = [Point(xy) for xy in zip(taxi_gdf['longitude'], taxi_gdf['latitude'])]

In [ ]:
taxi_gdf_final = gpd.GeoDataFrame(taxi_gdf, geometry='geometry')

In [ ]:
# Set the CRS to EPSG:4326 (WGS 84 Lat/Lon)
taxi_gdf_final = taxi_gdf_final.set_crs(epsg=4326)

In [ ]:
# Drop any locations that didn't have a matching zone
taxi_gdf_final = taxi_gdf_final.dropna(subset=['geometry'])

In [ ]:
print("\nFinal GeoDataFrame ready for export:")
print(taxi_gdf_final.head())

In [ ]:
# Export to GeoJSON 
output_filename = 'taxi_predictions_corrected.geojson'
taxi_gdf_final.to_file(output_filename, driver='GeoJSON')

You can now visualize in QGIS/ARGIS OR CARTO